# **Saylani Mass Training Program**
### **Cloud Data Engineering Module by Qasim Hassan**

#### A basice Extract, Transform and Load (ETL) pipeline using web scrapping, pandas and sql

#### Import necessary Libraries

In [32]:
#pip install pandas

In [33]:
from io import StringIO
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import sqlite3
from datetime import datetime
import logging

#### Step 0: Maintaining a Log File
This step is done to record the logs while performing ETL and it is not neccessary in an ETL Pipeline

In [34]:
import os
from datetime import datetime

def log_progress(message):
    """This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing.
    
    Also, it catches any errors during the logging process.
    """
    try:
        # Check if the 'logs' directory exists; if not, create it.
        if not os.path.exists('./logs'):
            os.makedirs('./logs')

        # Open the log file and write the log message.
        with open('./logs/code_log.txt', 'a') as f:
            f.write(f'{datetime.now()}: {message}\n')

    except Exception as e:
        # Handle any exceptions that occur during logging
        error_message = f"Logging error at {datetime.now()}: {str(e)}"
        try:
            # Attempt to log the error message into a separate error log file
            with open('./logs/error_log.txt', 'a') as error_f:
                error_f.write(f'{error_message}\n')
        except Exception as inner_e:
            # If error logging itself fails, print to the console as a last resort
            print(f"Failed to log error: {str(inner_e)}")

#### Step 1: Extract

In [35]:
url = "https://en.wikipedia.org/wiki/List_of_largest_banks"
table_attribs = 'By market capitalization'

soup = BeautifulSoup(requests.get(url).text, 'html.parser') # html format 
table = soup.find('span', string=table_attribs).find_next('table') # capital ka data lakar araha, table ki form may karrahi
df = pd.read_html(StringIO(str(table)))[0] # string then save in df 
df

,Rank,Bank name,Total assets (2023) (US$ billion)
0,1,Industrial and Commercial Bank of China,6303.44
1,2,Agricultural Bank of China,5623.12
2,3,China Construction Bank,5400.28
3,4,Bank of China,4578.28
4,5,JPMorgan Chase,3875.39
...,...,...,...
95,96,Raiffeisen Group,352.87
96,97,Handelsbanken,351.79
97,98,Industrial Bank of Korea,345.81
98,99,DNB,339.21


In [36]:
def extract(url, table_attribs):
    """ 
    This function extracts information from the website and returns a DataFrame. 
    """
    try:
        # Request the webpage and parse it
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the table and extract it into a DataFrame
        table = soup.find('span', string=table_attribs).find_next('table')
        df = pd.read_html(StringIO(str(table)))[0]

        log_progress('Data extraction complete.')

        return df

    except Exception as e:
        logging.error(f"Error: {e}")
        return None

In [37]:
#df

#### Step 2: Transform

In [38]:
# def transform(df, csv_path):
#     """ This function accesses the CSV file for exchange rate
#     information, and adds three columns to the data frame, each
#     containing the transformed version of Market Cap column to
#     respective currencies"""
def transform(df, csv_path):
    """ 
    This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies.
    """
    try:
        # Read the exchange rate CSV file
        exchange_rate = pd.read_csv(csv_path, index_col=0).to_dict()['Rate']

    except FileNotFoundError:
        print(f"Error: The file at {csv_path} was not found.")
        return None  # or handle as appropriate
    except pd.errors.EmptyDataError:
        print(f"Error: The file at {csv_path} is empty.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred while reading the file: {e}")
        return None

    try:
        # Perform transformation
        df['MC_GBP_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['GBP'], 2)
        df['MC_EUR_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['EUR'], 2)
        df['MC_INR_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['INR'], 2)
    except KeyError as e:
        print(f"Error: Missing currency rate for {e}.")
        return None  # or handle as appropriate
    except Exception as e:
        print(f"An error occurred during transformation: {e}")
        return None

    print(df)

    log_progress('Data transformation complete. Initiating Loading process')

    return df

#### Step 3: Load

Loading data to a CSV

In [39]:
df

,Rank,Bank name,Total assets (2023) (US$ billion)
0,1,Industrial and Commercial Bank of China,6303.44
1,2,Agricultural Bank of China,5623.12
2,3,China Construction Bank,5400.28
3,4,Bank of China,4578.28
4,5,JPMorgan Chase,3875.39
...,...,...,...
95,96,Raiffeisen Group,352.87
96,97,Handelsbanken,351.79
97,98,Industrial Bank of Korea,345.81
98,99,DNB,339.21


In [40]:
def load_to_csv(df, output_path):
    """ This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing."""

    df.to_csv(output_path)

    log_progress('Data saved to CSV file')

Loading data to SQL

In [41]:
def load_to_db(df, sql_connection, table_name):
    """ This function saves the final data frame to a database
    table with the provided name. Function returns nothing."""

    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

    log_progress('Data loaded to Database as a table, Executing queries')


def run_query(query_statement, sql_connection):
    """ This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. """

    cursor = sql_connection.cursor()
    cursor.execute(query_statement)
    result = cursor.fetchall()

    log_progress('Process Complete')

    print(result)

### Executing Pipeline

In [42]:
if __name__ == '__main__':
    url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    output_csv_path = 'output/Largest_banks_data.csv'
    database_name = 'Banks.db'
    table_name = 'Largest_banks'
    #
    log_progress('Preliminaries complete. Initiating ETL process')
    #

    df = (extract(url, 'By market capitalization'))

    transform(df, 'input/exchange_rate.csv')

    load_to_csv(df, output_csv_path)

    with sqlite3.connect(database_name) as conn:
        load_to_db(df, conn, table_name)

        print(run_query('SELECT * FROM Largest_banks', conn))

        print(run_query('SELECT AVG(MC_GBP_Billion) FROM Largest_banks', conn))

        print(run_query('SELECT "Bank name" FROM Largest_banks LIMIT 5', conn))

   Rank                                Bank name  Market cap (US$ billion)  \
0     1                           JPMorgan Chase                    432.92   
1     2                          Bank of America                    231.52   
2     3  Industrial and Commercial Bank of China                    194.56   
3     4               Agricultural Bank of China                    160.68   
4     5                                HDFC Bank                    157.91   
5     6                              Wells Fargo                    155.87   
6     7                        HSBC Holdings PLC                    148.90   
7     8                           Morgan Stanley                    140.83   
8     9                  China Construction Bank                    139.82   
9    10                            Bank of China                    136.81   

   MC_GBP_Billion  MC_EUR_Billion  MC_INR_Billion  
0          346.34          419.93        36798.20  
1          185.22          224.57    